Starting **May 7**, join weekly expert-led Q&A sessions to ask questions and get feedback from course lecturers and mentors.

[Sign up here](https://nebius.zoom.us/webinar/register/2017452395379/WN_fRQs-XofSzivwkNQUfGxBA) for the intro session — it's happening on May 7 at 7 PM CET.

# LLM Engineering Essentials 3.2. Database search and Vector Stores

In the previous notebook we used web search to provide additional context to LLMs, but often the relevant information doesn't just lie out there in the web - instead, it's contained in your company's internal databases.

In this notebook, you'll learn how to implement RAG with two popular database types: relational databases and vector stores. Another option - graph databases - will be considered in the next notebook.

<center>
<img src="https://drive.google.com/uc?export=view&id=1d4fUfWEYn6X1XW-B5VJNy5e5DZxZ30CC" width=600 />
</center>

**Note**. We'll still be using LLM APIs, but in many real-world situations this won't be a good option, because you don't want to expose your internal data to a third-party API. In such cases, self-hosted LLMs come to help; we'll learn how to use them further in the course.

## Getting things ready

In [1]:
!pip install -q openai

In [2]:
import os

with open("nebius_api_key", "r") as file:
    nebius_api_key = file.read().strip()

os.environ["NEBIUS_API_KEY"] = nebius_api_key

# Database search

In many applications, you need to supply the LLM with information about different product technicalities, which are stored in some internal database.



# Part 1: Text-to-SQL

In in many cases, the data source would be a **relational database**. Then, the **query tool** will have to reformulate the initial user's request as an SQL-like query. An example might be:

**User's request**: `Er, I'd like buy some potions, but I'm low on cash. What are the most affordable options?`

**SQL query**:

```sql
SELECT item_name, price
FROM shop_inventory
WHERE category = 'potion'
ORDER BY price ASC;
```

The task of generation of SQL queries from users' requests is known as **Text-to-SQL**. It's not an easy one! If you want to check how good LLMs and LLM-based systems are at it, we recomment to check the leaderboards of [BIRD-SQL Big Bench](https://bird-bench.github.io/) and [BIRD-CRITIC](https://bird-critic.github.io/). The second one is harder and only scores pure LLMs, not LLM-powered system, and it shows quite sad results; but even for BIRD-SQL AI systems are below 80% accuracy.

To compensate for this weakness, Text-to-SQL systems often introduce validating and debugging chains that improve the initial query until it's working. Here's, for example, a schematics describing the [Open Data QnA](https://github.com/GoogleCloudPlatform/Open_Data_QnA) framework by Google Cloud Plaftorm

<center>
<img src="https://raw.githubusercontent.com/GoogleCloudPlatform/Open_Data_QnA/refs/heads/main/utilities/imgs/OpenDataQnA_architecture.png" width=800 />

[Source](https://github.com/GoogleCloudPlatform/Open_Data_QnA)
</center>

We recommend you to check several top solutions from the BIRD leaderboard and take note of their system prompts. We'll only give general advise here. A typical Text-to-SQL prompt contains:

* A set of instructions, the more detailed, the better. Some instructions may explain the nature of the databases or some typical use cases; others set **restrictions**, usually forbidding the LLM from generating table-changing queries (INSERT, UPDATE, DELETE, DROP etc).

  Please note that even despite these restrictions, you should have additional post-LLM checks for that (based on regular expressions or other LLMs) to avoid your tables getting corrupted as a result of a hallucination or a jailbreal.

* Database schemas of all the necessary tables.

* If you can predict some frequent use cases beforehand, showing them in a system prompt won't also hurt.

Let's check an example!

## An example: a Potion Shop helper bot

We'll work with a simple SQL database, that has three tables:

```sql
CREATE TABLE shop_inventory (
  potion_id INTEGER PRIMARY KEY,      -- Unique ID of the potion
  stock INTEGER,                      -- How many are in stock
  price INTEGER                       -- Price in gold
)

CREATE TABLE potions (
  potion_id INTEGER PRIMARY KEY,      -- Matches inventory ID
  potion_name TEXT,                   -- Name of the potion
  category TEXT,                      -- Category (healing, mana, etc.)
  effect TEXT,                        -- Specific effect (heals 10 hp, etc.)
  rarity TEXT,                        -- common, uncommon, rare, legendary
  duration TEXT,                      -- How long it lasts (e.g., '1 min', '10 min', 'permanent')
  side_effects TEXT                   -- Possible side effects (nullable)
)

CREATE TABLE purchases (
  purchase_id INTEGER PRIMARY KEY,
  customer_name TEXT,                 -- Name of the customer
  potion_id INTEGER,                  -- Purchased potion
  quantity INTEGER,                   -- Number bought
  date DATE,                          -- Date of purchase
  FOREIGN KEY(potion_id) REFERENCES potions(potion_id)
)
```

The following code will download these databases along with some useful loader and query scripts.

In [3]:
!wget https://github.com/Nebius-Academy/LLM-Engineering-Essentials/raw/main/topic3/potion_shop.db -O potion_shop.db
!wget https://github.com/Nebius-Academy/LLM-Engineering-Essentials/raw/main/topic3/potion_shop_utils.py -O potion_shop_utils.py

--2025-04-23 01:56:23--  https://github.com/Nebius-Academy/LLM-Engineering-Essentials/raw/main/topic3/potion_shop.db
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/Nebius-Academy/LLM-Engineering-Essentials/main/topic3/potion_shop.db [following]
--2025-04-23 01:56:24--  https://raw.githubusercontent.com/Nebius-Academy/LLM-Engineering-Essentials/main/topic3/potion_shop.db
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16384 (16K) [application/octet-stream]
Saving to: ‘potion_shop.db’

potion_shop.db      100%[===================>]  16.00K  --.-KB/s    in 0s      

2025-04-23 01:56:24 (74.1 MB/s)

Let's load the database and run an SQL query to get three most expensive potions:

In [17]:
from potion_shop_utils import (
    create_potion_shop_database,
    show_schema,
    show_table,
    query_db
)

# Create the database
conn = create_potion_shop_database()

# Display schema
show_schema(conn)

# Display all tables
show_table(conn, "potions")
show_table(conn, "shop_inventory")
show_table(conn, "purchases")

# Example query
print("\nExample query: Most expensive potions")
result = query_db(conn, """
SELECT p.potion_name, s.price
FROM potions p
JOIN shop_inventory s ON p.potion_id = s.potion_id
ORDER BY s.price DESC
LIMIT 3
""")

for name, price in result:
    print(f"{name}: {price} gold")

Close the connection
conn.close()

Database already exists at potion_shop.db

== Database Schema ==

Table: shop_inventory
  potion_id (INTEGER) [PRIMARY KEY]
  stock (INTEGER)
  price (INTEGER)

Table: potions
  potion_id (INTEGER) [PRIMARY KEY]
  potion_name (TEXT)
  category (TEXT)
  effect (TEXT)
  rarity (TEXT)
  duration (TEXT)
  side_effects (TEXT)

Table: purchases
  purchase_id (INTEGER) [PRIMARY KEY]
  customer_name (TEXT)
  potion_id (INTEGER)
  quantity (INTEGER)
  date (DATE)

Table: potions
    potion_id                potion_name    category  \
0           1       Minor Healing Potion     healing   
1           2             Healing Potion     healing   
2           3       Major Healing Potion     healing   
3           4          Minor Mana Potion        mana   
4           5                Mana Potion        mana   
5           6          Major Mana Potion        mana   
6           7      Crude Barkskin Potion  protection   
7           8    Refined Barkskin Potion  protection   
8           9        

This seems correct.

Now, we'll create a class `TextToSQLRAGBot` implementing a RAG-powered bot that for each user's request:

* Creates an SQL query. For that, we use by default the **Phi-4** model my Microsoft, which is one of the top open source models at the [BIRD-CRITIC](https://bird-critic.github.io/) leaderboard.
* Runs this SQL query.
* Formulates a human-friendly answer based on the user's request and on whatever was retrieved from the database. For that, we use by default **Llama-3.1-70B**, but of course a much smaller model would work as well.

We decided that a simple scenario doesn't require implementing multi-turn conversations, so the bot has no conversation memory.

And first of all, let's create a system prompt for SLQ generation according to the guidelines discussed above. Note that it's not a system prompt in a strict sense, because the user's question will be a part of it as **{question}**:

In [5]:
text_to_sql_system_prompt = """**Task**

Generate a SQL query to answer the user's question. The data comes from a fantasy role-playing game shop system.

**Instructions**

- Generate a syntactically correct SQL query using only the schema provided.
- Do **not** select all columns with `SELECT *`; only select relevant columns.
- If the user requests sorted results, use `ORDER BY`—otherwise, avoid it.
- Do **not** return columns the user did not explicitly ask for.
- If a question cannot be answered with the available schema, return: `'I do not know'`.
- Do **not** use DML statements (INSERT, UPDATE, DELETE, DROP, etc.).
- Always use meaningful table aliases when joining multiple tables.
- You may assume `gold` is the unit of currency.

**Database Schema**

This query will run on a database with the following schema:

```sql
CREATE TABLE shop_inventory (
  potion_id INTEGER PRIMARY KEY,      -- Unique ID of the potion
  stock INTEGER,                      -- How many are in stock
  price INTEGER                       -- Price in gold
)

CREATE TABLE potions (
  potion_id INTEGER PRIMARY KEY,      -- Matches inventory ID
  potion_name TEXT,                   -- Name of the potion
  category TEXT,                      -- Category (healing, mana, etc.)
  effect TEXT,                        -- Specific effect (heals 10 hp, etc.)
  rarity TEXT,                        -- common, uncommon, rare, legendary
  duration TEXT,                      -- How long it lasts (e.g., '1 min', '10 min', 'permanent')
  side_effects TEXT                   -- Possible side effects (nullable)
)

CREATE TABLE purchases (
  purchase_id INTEGER PRIMARY KEY,
  customer_name TEXT,                 -- Name of the customer
  potion_id INTEGER,                  -- Purchased potion
  quantity INTEGER,                   -- Number bought
  date DATE,                          -- Date of purchase
  FOREIGN KEY(potion_id) REFERENCES potions(potion_id)
)
```

**Your Output**

Given the schema above, return the correct SQL query to answer this question:
**‘{question}’**

```sql
"""

In [6]:
from typing import Dict, Any, Optional, Callable
import sqlite3
import pandas as pd
import re

class TextToSQLRAGBot:
    def __init__(
        self,
        sql_client,  # Client for the SQL generation model
        response_client,  # Client for the response generation model
        database_connection: sqlite3.Connection,
        sql_model: str = "microsoft/phi-4",
        response_model: str = "meta-llama/Meta-Llama-3.1-70B-Instruct",
        get_sql_system_prompt = text_to_sql_system_prompt,
        get_response_system_prompt = None
    ):
        """Initialize the text-to-SQL RAG bot.

        Args:
            sql_client: Client for the SQL generation model (phi-4)
            response_client: Client for the response generation model (Llama-70b)
            database_connection: SQLite database connection
            sql_model: The model to use for SQL generation
            response_model: The model to use for response generation
            get_sql_system_prompt: Function to retrieve the system message for SQL generation
            get_response_system_prompt: Function to retrieve the system message for response generation
        """
        self.sql_client = sql_client
        self.response_client = response_client
        self.sql_model = sql_model
        self.response_model = response_model
        self.db_conn = database_connection

        self.get_sql_system_prompt = get_sql_system_prompt

        # Default system message for response generation if none is provided
        if get_response_system_prompt is None:
            self.get_response_system_prompt = lambda: """You are a helpful potion shop assistant that provides information based on database query results.

You will be given:
1. The original user question
2. The SQL query that was generated to answer the question
3. The results of executing that query on the potion shop database

Formulate a natural, helpful response that answers the user's question based on the query results.
Speak as if you are a knowledgeable potion shop employee helping a customer.
If the query returned no results, explain that to the user in a friendly way.
"""
        else:
            self.get_response_system_prompt = lambda: get_response_system_prompt

    def generate_sql_query(self, user_question: str) -> str:
        """Generate an SQL query from a natural language question.

        Args:
            user_question: The natural language question from the user

        Returns:
            str: The generated SQL query
        """

        try:
            # Get SQL query from the model
            completion = self.sql_client.chat.completions.create(
                model=self.sql_model,
                messages=[{
                    "role": "user",
                    "content": self.get_sql_system_prompt.format(question=user_question)
                }]
            )

            response = completion.choices[0].message.content

            # Look for SQL code blocks
            sql_blocks = re.findall(r"```sql\s*(.*?)\s*```", response, re.DOTALL)
            if sql_blocks:
                sql_query = sql_blocks[-1].strip()
            else:
                # Look for generic code blocks
                code_blocks = re.findall(r"```\s*(.*?)\s*```", response, re.DOTALL)
                if code_blocks:
                    sql_query = code_blocks[-1].strip()
                else:
                    # If no SQL-specific blocks, look for generic code blocks
                    # Which will most likely result in a failure
                    sql_query = response

            return sql_query

        except Exception as e:
            return f"Error generating SQL query: {str(e)}"

    def execute_query(self, sql_query: str) -> (pd.DataFrame, str):
        """Execute the SQL query on the database.

        Args:
            sql_query: The SQL query to execute

        Returns:
            tuple: (DataFrame with results, error message if any)
        """
        try:
            # Execute query and return results as a pandas DataFrame
            results = pd.read_sql_query(sql_query, self.db_conn)
            return results, None
        except Exception as e:
            return None, f"Error executing SQL query: {str(e)}"

    def generate_response(self, user_question: str, sql_query: str, query_results: pd.DataFrame) -> str:
        """Generate a natural language response based on the query results.

        Args:
            user_question: The original user question
            sql_query: The SQL query that was executed
            query_results: The results of the query as a pandas DataFrame

        Returns:
            str: The natural language response
        """
        messages = []
        system_prompt = self.get_response_system_prompt()
        if system_prompt:
            messages.append({
                "role": "system",
                "message": system_prompt
                })

        # Convert DataFrame to string representation
        if query_results is not None:
            results_str = query_results.to_string()
        else:
            results_str = "No results returned (query may have failed)"

        messages.append({
            "role": "user",
            "content": f"""User question: {user_question}

SQL query used:
{sql_query}

Query results:
{results_str}

Please provide a helpful response based on these results."""
        })

        try:
            # Get response from the model
            completion = self.response_client.chat.completions.create(
                model=self.response_model,
                messages=messages
            )

            response = completion.choices[0].message.content
            return response

        except Exception as e:
            return f"Error generating response: {str(e)}"

    def chat(self, user_question: str) -> Dict[str, Any]:
        """Process a user question end-to-end.

        Args:
            user_question: The natural language question from the user

        Returns:
            dict: Dictionary containing the original question, generated SQL, query results, and final response
        """
        # Step 1: Generate SQL query from user question
        sql_query = self.generate_sql_query(user_question)

        # Step 2: Execute the SQL query on the database
        query_results, error = self.execute_query(sql_query)

        # Step 3: Generate a natural language response based on the results
        if error:
            response = f"I'm sorry, but I couldn't execute the query. {error}"
        else:
            response = self.generate_response(user_question, sql_query, query_results)

        # Return a dictionary with all components of the process
        return {
            "user_question": user_question,
            "generated_sql": sql_query,
            "query_results": query_results,
            "response": response
        }

Now, let's create an instance of the RAG bot and run several simple SQL queries:

In [25]:
from openai import OpenAI
import sqlite3


client = OpenAI(
    base_url="https://api.studio.nebius.ai/v1/",
    api_key=os.environ.get("NEBIUS_API_KEY"),
)
sql_model = "microsoft/phi-4"
response_model = "meta-llama/Meta-Llama-3.1-70B-Instruct"

sql_client = client
response_client = client

# Create or load the database
db_conn = create_potion_shop_database()

# Initialize the Text-to-SQL RAG bot
text_to_sql_bot = TextToSQLRAGBot(
    sql_client=client,
    response_client=client,
    database_connection=db_conn,
    sql_model=sql_model,
    response_model=response_model
)

# Example questions to test the bot
example_questions = [
    "What are the most expensive potions?",
    "Which customer has spent the most money at the shop?",
    "How many healing potions do we have in stock?",
    "What potions has Alaric purchased?",
    "Which potions cause side effects related to dizziness or headaches?"
]

# Process each question
for i, question in enumerate(example_questions):
    print(f"\n--- Example {i+1}: {question} ---")

    result = text_to_sql_bot.chat(question)

    print("\nGenerated SQL:")
    print(result["generated_sql"])

    print("\nQuery Results:")
    if result["query_results"] is not None:
        print(result["query_results"])
    else:
        print("No results (query may have failed)")

    print("\nResponse:")
    print(result["response"])

    print("\n" + "="*60)



Database already exists at potion_shop.db

--- Example 1: What are the most expensive potions? ---

Generated SQL:
SELECT p.potion_name, s.price
FROM shop_inventory s
JOIN potions p ON s.potion_id = p.potion_id
ORDER BY s.price DESC;

Query Results:
                  potion_name  price
0     Mighty Berserker Elixir     70
1       Superior Speed Potion     65
2        Major Healing Potion     60
3     Dreamless Sleep Draught     50
4           Major Mana Potion     45
5     Refined Barkskin Potion     40
6   Frenzied Berserker Elixir     35
7          Swift Speed Potion     30
8              Healing Potion     25
9               Clarity Tonic     25
10                Mana Potion     20
11                   Antidote     20
12      Crude Barkskin Potion     15
13       Minor Healing Potion     10
14          Minor Mana Potion      8

Response:
The top 5 most expensive potions are:

1. Mighty Berserker Elixir ($70)
2. Superior Speed Potion ($65)
3. Major Healing Potion ($60)
4. Dreamless S

Feel free to check if our bot answered correctly!

Now, a less successful example:

In [29]:
result = text_to_sql_bot.chat("""
Today is 2024-05-01 (May 1st, 2024)
How many of each of the following potions we had on 2024-02-01
if we didn't have resupply since January:
- Minor Healing Potion
- Major Mana Potion
- Swift Speed Potion?
""")

In [31]:
print(result["generated_sql"])

SELECT 
    pot.potion_name,
    (inv.stock - COALESCE(SUM(pur.quantity), 0)) AS remaining_stock
FROM 
    potions pot
JOIN 
    shop_inventory inv ON pot.potion_id = inv.potion_id
LEFT JOIN 
    purchases pur ON pot.potion_id = pur.potion_id
    AND pur.date < '2024-02-01'
WHERE 
    pot.potion_name IN ('Minor Healing Potion', 'Major Mana Potion', 'Swift Speed Potion')
GROUP BY 
    pot.potion_name, inv.stock;


Here the problem is that instead of adding all the purcases since 2024-02-01 to the current stock, the LLM suggests subtracting from today's stock the purchases that happened *before* 2024-02-01. This could be addressed with a more powerful LLM: Claude copes well with this request.

In [39]:
# Uncomment this to close the database connection when done
# db_conn.close()

## Beyond SQL

Relational databases are great for working with structured data. However, sometimes you just need to store a pile of texts, or code, or JSONs, or, worse even, images or videos. Let's see how to deal with them!

# Part 2: Vector Databases

We can store whatever we want in a `(key, value)` format as long as the keys are convenient for retrieval. And it turns out that vectors are really good in this role!

So, a **vector database** (or a **vector store**) stores data in key-value format with **vectors for keys**. Let's briefly discuss the overall pipeline of vector database usage.

## Encoders and embeddings

To put an object into a vector store, you must **encode** it as a vector, or we might also say, you construct its **(vector) embedding**. This is done with a dedicated **encoder**, which is usually a neural network of sorts. You can learn more about **text encoders** in the [dedicated long read](https://nebius-academy.github.io/knowledge-base/text-encoders/). There are also a number of popular embedding models for images and videos (soon, we'll make a long read about them too).

It's very important to point out that a good encoder doesn't just transform objects into vectors in some random way. Rather, the crucial feature for a good encoder is that **it maps semantically close objects to geometrically close vectors**. Thus, in a picture below, the vectors representing orcs cluster together, while the vectors representing cats are in a distance from them.

<center>
<img src="https://drive.google.com/uc?export=view&id=14m2FfYjE_XUXGJkc4D8w4zzAlxVNVBrb" width=600 />
</center>

As a measure of geometric proximity, either **Euclidean distance** is used or **cosine similarity**, while sometimes just the scalar product $(x, y) = x^Ty = x_1y_1+\ldots+x_dy_d$ is used because of its computational simplicity.

<center>
<img src="https://drive.google.com/uc?export=view&id=1srHjEQuiGPoLaz9KEt8IXRxZol2mJjqG" width=600 />
</center>

## Retrieval in vector stores

Retrieval in a vector store relies on the **Nearest Neighbor Search** mechanism. A query to a vector database is an object like those stored in that database. What happens with the query is:

1. First, the query is vectorized by the encoder to get its embedding.
2. Then, the database finds the prescribed number of its nearest neighbor vectors.
3. Finally, the database returns the objects corresponding to these vectors.

<center>
<img src="https://drive.google.com/uc?export=view&id=1ZeUSt-_VsBRZtHCfFNzde6iwiM9ltBgx" width=600 />
</center>

**Note**. Models such as **CLIP** are able to encode both texts and images in a coherent way, mapping semantically related texts and images to geometrically close vectors. With such a model, you can, for example, have a database of images and query it with a text. You'll try this in the practice part.

Of course, efficient retrieval requires quick nearest neighbour search, and for large databases or/and low-latency scenarios it's not a good idea to just cycle through all the items looking for the nearest one. So, there is a number of optimized strategies that pre-compute some additional data structures which allow for faster (even if approximate) neighbour search. One of the most popular ones is **HNSW** (**Hierarchical Navigable Small World**). If you're curious, we have a short explanation and a small demo further in this notebook.

## Vector store zoo

A huge variety of vector databases are available now, including [Chroma](https://docs.trychroma.com/docs/overview/getting-started), [LanceDB](https://lancedb.github.io/lancedb/), [Weaviate](https://weaviate.io/developers/weaviate), [Qdrant](https://qdrant.tech/documentation/) and many, many more - and choosing one is not a simple task. When you have less than several million vectors to store, it's actually more a question of a developer's convenience. While, at larger scales, the speed of pre-computation and nearest neighbor search may start contributing significantly to the overall efficiency.

If you need to choose a vector store for your project, you can start by checking [this comparison chart](https://superlinked.com/vector-db-comparison), or a similar one.

## RAG with LanceDB demo

In this demo we'll apply RAG to perform Q&A about the `transformers` library using this library's documentation.

In [2]:
!pip install -q openai

In [7]:
import os

with open("nebius_api_key", "r") as file:
    nebius_api_key = file.read().strip()

os.environ["NEBIUS_API_KEY"] = nebius_api_key

Let's get the docs.

### Text preprocessing

Embedding magic won't work properly if the text is cluttered with formatting artifacts. Unless the formatting carries meaning (as it does in code), it's generally best to remove it before encoding.

So, we're going to strip markdown from most of its structure leaving only plain text.

In [8]:
import os
import re

from tqdm import tqdm
from bs4 import BeautifulSoup
from markdown import markdown
from pathlib import Path


def markdown_to_text(markdown_string):
    """ Converts a markdown string to plaintext """

    # md -> html -> text since BeautifulSoup can extract text cleanly
    html = markdown(markdown_string)

    html = re.sub(r'<!--((.|\n)*)-->', '', html)
    html = re.sub('<code>bash', '<code>', html)

    # extract text
    soup = BeautifulSoup(html, "html.parser")
    text = ''.join(soup.findAll(text=True))

    text = re.sub('```(py|diff|python)', '', text)
    text = re.sub('```\n', '\n', text)
    text = re.sub('-         .*', '', text)
    text = text.replace('...', '')
    text = re.sub('\n(\n)+', '\n\n', text)

    return text


def prepare_files(input_dir="transformers/docs/source/en/", output_dir="docs"):
    # Convert string paths to Path objects
    input_dir = Path(input_dir)
    output_dir = Path(output_dir)

    # Check if input directory exists
    assert input_dir.is_dir(), "Input directory doesn't exist"
    output_dir.mkdir(parents=True, exist_ok=True)

    for root, subdirs, files in tqdm(os.walk(input_dir)):
        root_path = Path(root)
        for file_name in files:
            file_path = root_path / file_name
            parent = root_path.stem if root_path.stem != input_dir.stem else ""

            if file_path.is_file():
                with open(file_path, encoding="utf-8") as f:
                    md = f.read()
                text = markdown_to_text(md)

                output_file = output_dir / f"{parent}_{Path(file_name).stem}.txt"
                with open(output_file, "w", encoding="utf-8") as f:
                    f.write(text)


In [4]:
!git clone https://github.com/huggingface/transformers

Cloning into 'transformers'...
remote: Enumerating objects: 278088, done.
remote: Counting objects: 100% (133/133), done.
remote: Compressing objects: 100% (95/95), done.
remote: Total 278088 (delta 80), reused 40 (delta 36), pack-reused 277955 (from 3)
Receiving objects: 100% (278088/278088), 289.12 MiB | 25.67 MiB/s, done.
Resolving deltas: 100% (206671/206671), done.


In [9]:
prepare_files()

0it [00:00, ?it/s]<ipython-input-8-d285d05c636f>:21: DeprecationWarning: Call to deprecated method findAll. (Replaced by find_all) -- Deprecated since version 4.0.0.
  text = ''.join(soup.findAll(text=True))
6it [00:07,  1.18s/it]


### Chunking

Text encoders may struggle with a long, information-rich text fragment. To make your encoder's life easier (and your RAG results better), you need to split your context into relatively small meaningful chunks.

That said, there is no "golden" chunking strategy. (Ideally, each chunk should be a wholesome content piece dedicated to a single topic, but it rarely works this way.) By and large, one of the following two (simple) strategies are used:

1. **Fixed token size with overlap**. This is a very straightforward approach, as it is likely to trip, crop, and split useful information. By overlapping chunks, we ensure that important information at the boundaries of each chunk is captured in multiple chunks, helping to mitigate the losses and maintain context continuity.

Note that a chunk size is usually several hundreds of tokens. You can start with 512 and then tune it for your task if needed.

1. **Recursive chunking**. First of all, this requires setting maximal chunk length in tokens. Now, as an example, you could split a plain text document like this:
    - First, chunk by '\n\n' (that end of a "section").
    - Then, if a "section" is too long, by '\n' (by paragraph).
    - Then by '.' (by sentence).
    - At this point, if you're still above maximal length, just split by a fixed token length.

Note that the separator sequence ['\n\n', '\n', '.'] may be replaced by any other fit for your data. (Langchain's Recursive Character Text Splitter implements precisely this strategy.)

Of course, you can use other specific features of your data to chunk it more meaningfully. For example, if you're dealing with markdown, it's wise to use specific separators for Recursive Chunking. To illustrate, in LangChain's Markdown Text Splitter, the following list is used:

```
[
    "\\n#{1,6} ",
    "```\\n",
    "\\n\\*\\*\\*+\\n",
    "\\n---+\\n",
    "\\n___+\\n",
    "\\n\\n",
    "\\n",
    " ",
    "",
]

```

The final "" indicates that we've just split by a fixed token length specified by the user at the setup of the Text Splitter tool if all separators are depleted.

A good practice is to assess your chunks manually in order to check if their size is OK and that they stay meaningful under your chosen splitting strategy.

We will perform chunking using recursive text splitter from the `Langchain` library.

In [ ]:
!pip install lancedb pyarrow tiktoken -q
!pip install -qU langchain-text-splitters

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 67.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 59.6 MB/s eta 0:00:00


In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    separators=[
        "\n\n",
        "\n",
        ".",
        " "
    ],
    chunk_size=1024,
    chunk_overlap=128,
    length_function=len,
    is_separator_regex=False,
)

In [ ]:
import os
from typing import List
from functools import partial

import lancedb
from lancedb.pydantic import LanceModel, Vector
from lancedb.embeddings import get_registry

### Creating a database

Let's create a LanceDB database. For that, we'll need to define several things:

1. The **embedding model**. It's set up with
  
  `embed_func = get_registry().get(<provider name>).create(name=<embedding name>)`
  
  For example,
  
  `embed_func = get_registry().get("openai").create(name="text-embedding-ada-002")`

  We'll use a free model from Hugging Face for demonstration.

2. The **database entry schema**. At the very least it should contain:
  
  * The **vector** field
  * The **field which is vectorized**, in our case `text`. It's marked with `embed_func.SourceField()`

In [ ]:
# This line is needed in case you've ran this cell before to clear the db dir
!rm -rf /tmp/lancedb

db = lancedb.connect("/tmp/lancedb")

In [ ]:
import openai
import pyarrow as pa

# We use this model as the encoder: https://huggingface.co/BAAI/bge-small-en-v1.5
embed_func = get_registry().get("huggingface").create(name="BAAI/bge-small-en-v1.5")


class BasicSchema(LanceModel):
    '''
    This is how we store data in the database.
    We need to have a vector here, but apart from this, we may have many other fields
    '''
    text: str = embed_func.SourceField()
    vector: Vector(embed_func.ndims()) = embed_func.VectorField(default=None)

lance_table = db.create_table(
    "transformer_docs",
    mode='overwrite',
    schema=BasicSchema
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/133M [00:00<?, ?B/s]

Let's populate the database:

In [ ]:
from tqdm import tqdm
splitted_docs = []

for file in tqdm(os.listdir("docs")):
    with open("docs/"+file, "r") as f:
        text = f.read()
        docs = text_splitter.create_documents([text])
        splitted_docs.extend([{"text": doc.page_content} for doc in docs])

100%|██████████| 514/514 [00:00<00:00, 5246.10it/s]


In [ ]:
print("Total splits:", len(splitted_docs))
print("==First split:==\n", splitted_docs[0])
print("==Second split:==\n", splitted_docs[1])

Total splits: 4423
==First split:==
 {'text': 'Processors\nMultimodal models require a preprocessor capable of handling inputs that combine more than one modality. Depending on the input modality, a processor needs to convert text into an array of tensors, images into pixel values, and audio into an array with tensors with the correct sampling rate.\nFor example, PaliGemma is a vision-language model that uses the SigLIP image processor and the Llama tokenizer. A [ProcessorMixin] class wraps both of these preprocessor types, providing a single and unified processor class for a multimodal model.\nCall [~ProcessorMixin.from_pretrained] to load a processor. Pass the input type to the processor to generate the expected model inputs, input ids and pixel values.'}
==Second split:==
 {'text': 'from transformers import AutoProcessor, PaliGemmaForConditionalGeneration\nfrom PIL import Image\nimport requests\nprocessor = AutoProcessor.from_pretrained("google/paligemma-3b-pt-224")\nprompt = "answe

Populating the table may take some time:

In [ ]:
lance_table.add(
    splitted_docs,
    on_bad_vectors='drop'  # or 'fill' with fill_value=0.0
)

We'll create a `search_table` function that will allow us to fetch up to `limits` documents from the database for our `query`:

In [ ]:
def search_table(table, query, max_results=5):
    return table.search(query).limit(max_results).to_pydantic(BasicSchema)

In [ ]:
result = search_table(lance_table, "How to load an LLM in 4 bit quantization?", max_results=5)
result

[BasicSchema(text='Set up a [BitsAndBytesConfig] and set load_in_4bit=True to load a model in 4-bit precision. The [BitsAndBytesConfig] is passed to the quantization_config parameter in [~PreTrainedModel.from_pretrained].\nAllow Accelerate to automatically distribute the model across your available hardware by setting device_map=“auto”.\nPlace all inputs on the same device as the model.\n\nfrom transformers import BitsAndBytesConfig, AutoTokenizer, AutoModelForCausalLM\nquantization_config = BitsAndBytesConfig(load_in_4bit=True)\ntokenizer = AutoTokenizer("meta-llama/Llama-3.1-8B")\nmodel = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.1-8B", device_map="auto", quantization_config=quantization_config)\nprompt = "Hello, my llama is cute"\ninputs = tokenizer(prompt, return_tensors="pt").to(model_8bit.device)\ngenerated_ids = model_8bit.generate(**inputs)\noutputs = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)', vector=FixedSizeList(dim=384)),
 BasicSchema(te

In [ ]:
result = search_table(lance_table, "LLM in 4 bit quantization", max_results=5)
result

[BasicSchema(text='bitsandbytes\nbitsandbytes features the LLM.int8 and QLoRA quantization to enable accessible large language model inference and training.\nLLM.int8() is a quantization method that aims to make large language model inference more accessible without significant degradation. Unlike naive 8-bit quantization, which can result in loss of critical information and accuracy, LLM.int8() dynamically adapts to ensure sensitive components of the computation retain higher precision when needed.\nQLoRA, or 4-bit quantization, compresses a model even further to 4-bits and inserts a small set of trainable low-rank adaptation (LoRA) weights to allowing training. \n\nNote: For a user-friendly quantization experience, you can use the bitsandbytes community space.\n\nRun the command below to install bitsandbytes.', vector=FixedSizeList(dim=384)),
 BasicSchema(text='Quantization Fundamentals with Hugging Face\nQuantization in Depth\nIntroduction to Quantization cooked in 🤗 with 💗🧑\u200d🍳\

As you see, the search results may vary as you reformulate the query.

### Answering questions with RAG

Finally, it's time to generate something!

In the previous notebook the `answer_with_db` function used web search; we'll update it to use database search instead.

In [ ]:
from openai import OpenAI
import os

nebius_client = OpenAI(
    base_url="https://api.studio.nebius.ai/v1/",
    api_key=os.environ.get("NEBIUS_API_KEY"),
)
llama_8b_model = "meta-llama/Meta-Llama-3.1-8B-Instruct"

def prettify_string(text, max_line_length=80):
    """Prints a string with line breaks at spaces to prevent horizontal scrolling.
    Args:
        text: The string to print.
        max_line_length: The maximum length of each line.
    """
    output_lines = []
    lines = text.split("\n")
    for line in lines:
        current_line = ""
        words = line.split()
        for word in words:
            if len(current_line) + len(word) + 1 <= max_line_length:
                current_line += word + " "
            else:
                output_lines.append(current_line.strip())
                current_line = word + " "
        output_lines.append(current_line.strip())  # Append the last line
    return "\n".join(output_lines)

def search_result_to_context(search_result):
    return "\n\n".join(
        [record.text for record in search_result]
    )

def answer_with_rag(
    prompt: str,
    system_prompt=None,
    max_tokens=512,
    client=nebius_client,
    model=llama_8b_model,
    table=None,
    prettify=True,
    temperature=0.6,
    max_results=5,
    verbose=False
) -> str:
    """
    Generate an answer using RAG (Retrieval-Augmented Generation) with database search.

    Args:
        prompt: User's question or prompt
        system_prompt: Instructions for the LLM
        max_tokens: Maximum number of tokens in the response
        client: OpenAI client instance
        model: Model identifier
        table: LanceDB table
        prettify: Whether to format the output text
        temperature: Temperature for response generation
        max_results: Maximal number of documents to fetch from the table
        verbose: whether to return the search results as well

    Returns:
        Generated response incorporating search results
    """
    # Perform database search
    if table:
        try:
            search_results = search_table(table, prompt, max_results=max_results)
        except:
            search_results = []
    else:
        search_results = []

    # Construct messages with search results
    messages = []

    if system_prompt:
        messages.append({
            "role": "system",
            "content": system_prompt
        })

    # Add user prompt
    messages.append({
        "role": "user",
        "content":
            f"""Answer the following query using the context provided.

            <context>\n{search_results}\n</context>

            <query>{prompt}</query>
            """
    })

    # Generate completion
    completion = client.chat.completions.create(
        model=model,
        messages=messages,
        max_tokens=max_tokens,
        temperature=temperature
    )

    if prettify:
        answer = prettify_string(completion.choices[0].message.content)
    else:
        answer = completion.choices[0].message.content

    if verbose:
        return {
            "answer": answer,
            "search_results": search_results
        }
    else:
        return answer

Now, let's first try asking Llama how to load an LLM in 4 bit quantization without supplying context (`table=None`).

In [ ]:
client = OpenAI(
    base_url="https://api.studio.nebius.ai/v1/",
    api_key=os.environ.get("NEBIUS_API_KEY"),
)
model = "meta-llama/Meta-Llama-3.1-8B-Instruct"

results = answer_with_rag("""How to load an LLM in 4 bit quantization?""",
               client=client, model=model, table=None, verbose=True)
print(results["answer"])

Unfortunately, there is no context provided to give a specific answer to the
query. However, I can provide a general outline on how to load a Large Language
Model (LLM) in 4-bit quantization.

**4-bit Quantization**

4-bit quantization is a technique used to reduce the precision of neural
network weights and activations from 32-bit floating-point numbers to 4-bit
integers. This reduces the memory footprint and computational requirements of
the model.

**Loading an LLM in 4-bit Quantization**

To load an LLM in 4-bit quantization, you'll need to follow these general
steps:

1. **Quantize the LLM weights**: Use a quantization library or framework (e.g.,
TensorFlow, PyTorch, or ONNX) to convert the LLM weights from 32-bit
floating-point numbers to 4-bit integers. This involves mapping the weight
values to a smaller range of values that can be represented by 4-bit integers.
2. **Choose a quantization strategy**: Select a quantization strategy, such as:
* **Weight-wise quantization**: Quant

As you see, the answer is vague and not qualified. Now, let's add proper context!

In [ ]:
client = OpenAI(
    base_url="https://api.studio.nebius.ai/v1/",
    api_key=os.environ.get("NEBIUS_API_KEY"),
)
model = "meta-llama/Meta-Llama-3.1-8B-Instruct"

results = answer_with_rag("""How to load an LLM in 4 bit quantization?""",
               client=client, model=model, table=lance_table, verbose=True)
print(results["answer"])

To load an LLM in 4-bit quantization, you can use the `BitsAndBytesConfig`
class from the `transformers` library. Here is an example of how you can do it:

```python
from transformers import AutoModelForCausalLM, AutoTokenizer,
BitsAndBytesConfig

# Load the model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.1-8B")
model = AutoModelForCausalLM.from_pretrained(
"meta-llama/Llama-3.1-8B",
quantization_config=BitsAndBytesConfig(load_in_4bit=True),
device_map="auto"
)
```

In this example, the `BitsAndBytesConfig` class is used to specify the
quantization configuration. The `load_in_4bit=True` parameter tells the model
to load in 4-bit precision. The `device_map="auto"` parameter allows the model
to automatically distribute itself across the available hardware.

Note that you need to make sure that the model you are loading is compatible
with 4-bit quantization. Also, the performance benefits of 4-bit quantization
may vary depending on the specific hardware a

That's much better!

In [ ]:
results["search_results"]

[BasicSchema(text='Set up a [BitsAndBytesConfig] and set load_in_4bit=True to load a model in 4-bit precision. The [BitsAndBytesConfig] is passed to the quantization_config parameter in [~PreTrainedModel.from_pretrained].\nAllow Accelerate to automatically distribute the model across your available hardware by setting device_map=“auto”.\nPlace all inputs on the same device as the model.\n\nfrom transformers import BitsAndBytesConfig, AutoTokenizer, AutoModelForCausalLM\nquantization_config = BitsAndBytesConfig(load_in_4bit=True)\ntokenizer = AutoTokenizer("meta-llama/Llama-3.1-8B")\nmodel = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.1-8B", device_map="auto", quantization_config=quantization_config)\nprompt = "Hello, my llama is cute"\ninputs = tokenizer(prompt, return_tensors="pt").to(model_8bit.device)\ngenerated_ids = model_8bit.generate(**inputs)\noutputs = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)', vector=FixedSizeList(dim=384)),
 BasicSchema(te

# Practice part

If you encounter any difficulties or simply want to see our solutions, feel free to check the [Solutions notebook](https://colab.research.google.com/github/Nebius-Academy/LLM-Engineering-Essentials/blob/main/topic3/3.2_database_search_and_vector_stores_solutuons.ipynb).

## Task 1. A simple recommender system

Vector search is a reasonable baseline for either retrieval or recommendations. In this task, we'll test it on a simple database containing information about goods that might be sold by a fantasy trader. (Of course, those could be any items an online store would sell.)

Let's download the data.

In [1]:
!wget https://github.com/Nebius-Academy/LLM-Engineering-Essentials/raw/main/topic3/fantasy_items_descriptions.json -O fantasy_items_descriptions.json

--2025-04-19 01:35:38--  https://github.com/Nebius-Academy/LLM-Engineering-Essentials/raw/main/topic3/fantasy_items_descriptions.json
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/Nebius-Academy/LLM-Engineering-Essentials/main/topic3/fantasy_items_descriptions.json [following]
--2025-04-19 01:35:38--  https://raw.githubusercontent.com/Nebius-Academy/LLM-Engineering-Essentials/main/topic3/fantasy_items_descriptions.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 361418 (353K) [text/plain]
Saving to: ‘fantasy_items_descriptions.json’

fantasy_items_descr 100%[===================>] 352.95K  

In [7]:
import json

with open('fantasy_items_descriptions.json', 'r') as f:
    data = json.load(f)

data[0]

{'name': 'Enhanced Minor Healing Potion',
 'category': 'Potion',
 'rarity': 'Uncommon',
 'structured': {'name': 'Enhanced Minor Healing Potion',
  'category': 'Potion',
  'rarity': 'Uncommon',
  'appearance': 'A delicate glass vial filled with a rich, golden liquid that shimmers softly in the light and has a faint scent of honey and rose petals.',
  'effect': 'Restores a moderate amount of health (typically 2d8 + 2 HP or equivalent) and grants a slight boost to vitality, allowing the drinker to ignore the first point of damage they take in the next hour.',
  'usage_instructions': 'Drink the entire contents of the vial slowly, feeling the soothing warmth spread through the body.',
  'duration': 'Instant healing, with the vitality boost lasting for 1 hour.',
  'side_effects': 'A gentle tingling sensation in the fingers and toes, and a subtle feeling of rejuvenation.',
  'origin_lore': 'Brewed by skilled herbalists who infuse the mixture with prayers to the gods of healing and protection,

As you see, we have quite a lot of information about every item. It is presented it two ways:

* structured info in the `structured` field
* text description which retells the same information in a playful and somewhat distorted wat in the `vague_description` field.

Now, your task will be to create a two-stage recommender system, that for a user's query will:

* First, perform a database search to find 5-10 candidates,
* Then, query an LLM to analyze the search results and choose 2-3 final candidates. This way, the LLM will play the role of a **reranker**. We'll talk more about rerankers in the next notebook.

We also encourage you to do the following experiments:

* Try and compare retrieval that uses either of the three databases:
  
  * A database consisting of only vague descriptions
  * A database containing structured descriptions in JSON format
  * A database containing structured descriptions in pure text format, with all JSON markup stripped

* Try and compare retrieval with and without query preprocessing. In such scenarios it's often beneficial to reformulate a query, making it less colloquial and more to the point

We'll discuss automatic RAG scoring in one of the following weeks. Right now, we suggest just gathering 10 diverse queries and looking closely at the outputs.

In [ ]:
# <YOUR CODE HERE>

## Task 2. Trying another vector database - an exercise in LLM-assisted development

There are many vector stores, and you might need to use different ones depending on the tastes of your colleagues and various other considerations. In this task, you'll try [Qdrant](https://qdrant.tech/), which is also a convenient and efficient database.

We suggest that you use an in-memory client for this exercise.

And since we don't provide any explanations, the best way of coping with this task will be to ask an LLM to translate the above code from LanceDB to Qdrant! Our advice is using an LLM that can also perform web search (ChatGPT or Gemini, for example), because the libraries may change quite rapidly. This shouldn't be problematic, but if you struggle with the task, feel free to check our solution. Be prepared though that the code might fail once or twice before you fix it with the help of the LLM.

We'll start with the usual preparations.

In [11]:
import os
import re

from tqdm import tqdm
from bs4 import BeautifulSoup
from markdown import markdown
from pathlib import Path


def markdown_to_text(markdown_string):
    """ Converts a markdown string to plaintext """

    # md -> html -> text since BeautifulSoup can extract text cleanly
    html = markdown(markdown_string)

    html = re.sub(r'<!--((.|\n)*)-->', '', html)
    html = re.sub('<code>bash', '<code>', html)

    # extract text
    soup = BeautifulSoup(html, "html.parser")
    text = ''.join(soup.findAll(text=True))

    text = re.sub('```(py|diff|python)', '', text)
    text = re.sub('```\n', '\n', text)
    text = re.sub('-         .*', '', text)
    text = text.replace('...', '')
    text = re.sub('\n(\n)+', '\n\n', text)

    return text


def prepare_files(input_dir="transformers/docs/source/en/", output_dir="docs"):
    # Convert string paths to Path objects
    input_dir = Path(input_dir)
    output_dir = Path(output_dir)

    # Check if input directory exists
    assert input_dir.is_dir(), "Input directory doesn't exist"
    output_dir.mkdir(parents=True, exist_ok=True)

    for root, subdirs, files in tqdm(os.walk(input_dir)):
        root_path = Path(root)
        for file_name in files:
            file_path = root_path / file_name
            parent = root_path.stem if root_path.stem != input_dir.stem else ""

            if file_path.is_file():
                with open(file_path, encoding="utf-8") as f:
                    md = f.read()
                text = markdown_to_text(md)

                output_file = output_dir / f"{parent}_{Path(file_name).stem}.txt"
                with open(output_file, "w", encoding="utf-8") as f:
                    f.write(text)


And now it's your turn to ingest the data into a Qdrant database instance and to update the `answer_with_rag` function accordingly!

In [ ]:
# <YOUR CODE HERE>

## Task 3. A safer text2sql device

When you combine SQL and LLMs, you might face the challenge of determining the amount of freedom you're ready to provide to the agent. Can it only execute `SELECT` queries? Or do you allow it to modify your tables in certain ways? The more freedom you give it, the more you risk losing data or getting it changed in unpredictable ways.

Our `text_to_sql_bot` uses

```
pd.read_sql_query(sql_query, self.db_conn)
```

to run queries. This function is designed to execute queries that return rows (which are mostly `SELECT` queries) and it will fail to execute a query that manipulates with tables or data (so, no `DROP`, `DELETE`, `CREATE` etc).

But what if our SQL bot actually needed to alter the table? For example, to edit side effects based on customers' feedback?

In this task, you'll give it a try — and you might get a little bruised along the way. What we suggest you to do:

**Step 1.** Try to update the bot, introducing as few ad-hoc changes as possible. Namely:

- Change `pd.read_sql_query(sql_query, self.db_conn)` into `query_db(self.db_conn, sql_query)` and relax the system prompt guidelines a bit to allow for some changes - namely for adding side effects to potions.

**Step 2.** No matter what you write in the system prompt, you've actually invited havoc into your life when you allowed execution of random queries. Test it by making the bot delete data about Thorne's purchases. Thorne is our top-spending customer; you can use it ;)

Recall the jailbreaking techniques from Topic 1. Execise all you guile and ingenuity! It won't be that complicated in the end.

**Step 3.** Now, try to make the bot safe again. The basic principle here is: you can't forbid an LLM from doing anything wiht just prompting. So, if you want a really safe system, you'll need to do some hardcoding. Here are some possible solutions you could explore:

* Introducing simple checks that the `sql_query` generated by the LLM doesn't contain bad words such as `DELETE`, `DROP` etc.
* Adding on top of that a layer of LLM defense: for non-SELECT queries it would decide whether it's benign or malicious. Again, no LLM is 100% failure-proof, but tricking an LLM whose input and output are concealed from you might be way more complicated.
* Instead of performing checks after generating the SQL query, you can make it in the very beginning. Just add an LLM classifier to discern between two situations:

  - A potential `SELECT` statement for which the safe `pd.read_sql_query(sql_query, self.db_conn)` will be called
  - A request for adding a specific side effect to a specific potion - these could be obtained with JSON outputs. Then, you'll just plug them into a ready-made SQL query template!

This all doesn't seem too much exciting - it's almost as if we don't believe in the magic of LLMs... But the thing is: magic is cool but only as long as your buiseness processes can't get hurt by its side effects.

So be safe and take the power of LLMs with a grain of salt!

In [ ]:
# <YOUR CODE HERE>